## Machine learning intro

![](https://newapplift-production.s3.amazonaws.com/comfy/cms/files/files/000/001/201/original/machine-learning-robots-dilbert.gif) 

### Loss function

Training and quality assessment of the model is carried out on independent sets of examples. As a rule, existing examples are divided into two subsets: training (train) and test (test). The choice of split ratio is a compromise. Indeed, the large size of the training leads to better algorithms, but more noise when evaluating the model on the test. Conversely, a large test sample size leads to a less noisy quality assessment, but the trained models are less accurate.

Many classification models predict a rating of membership to a positive class $ \tilde {y} (x) \in R $ (for example, the probability of being a class 1). After that, a decision is made on the class of the object by comparing the estimate with a certain threshold $ \ theta $:

$$y(x) = 
\begin{cases}
+1, &\text{if} \; \tilde{y}(x) \geq \theta \\
-1, &\text{if} \; \tilde{y}(x) < \theta
\end{cases}
$$

In this case, we can consider metrics that can work with the initial response of the classifier. In the assignment, we will work with the AUC-ROC metric, which in this case can be considered as the proportion of incorrectly ordered pairs of objects sorted by ascending the predicted grade 1 grade. A detailed understanding of how the AUC-ROC metrics work for this lab is not required.

### Selection of hyperparameters of the model

In machine learning tasks, one should distinguish between model parameters and hyperparameters (structural parameters). Typically, model parameters are adjusted during training (for example, weights in a linear model or a decision tree structure), while hyper parameters are set in advance (for example, the value of the regularization force in a linear model or the maximum depth of a decision tree). Each model, as a rule, has a lot of hyperparameters and there are no universal sets of hyperparameters that work optimally in all tasks, so for each task you need to choose your own set.

To optimize the hyperparameters, models often use _bind on the grid (grid search) _: for each hyperparameter, several values are selected, then all combinations of values are selected and a combination is selected on which the model shows the best quality (in terms of the metric being optimized). However, in this case, it is necessary to correctly evaluate the constructed model, namely, to do the partition into a training and test sample. There are several schemes for how this can be implemented:

 - Break the available sample into training and test. In this case, the comparison of a large number of models in the enumeration of hyperparameters leads to a situation where the best model on the test subsample does not retain its qualities on the new data. We can say that there is a _transition_ on a test sample.
 - To eliminate the problem described above, you can split the data into 3 non-overlapping subsamples: training, validation, and test. Validation subsample is used to compare models, and test - for the final quality assessment and comparison of families of models with selected hyperparameters.
 - Another way to compare models is [cross-validation] (http://bit.ly/1CHXsNH). There are various cross-validation schemes:
  - Leave-One-Out
  - K-Fold
  - Repeated random sampling
  
Cross validation is computationally expensive, especially if you are iterating over a grid with a very large number of combinations. Given the finiteness of the time to perform the task, a number of compromises arise: 
  - the hyperparameter grid can be made more sparse by going through fewer values of each hyperparameter; however, you should not forget that in this case you can skip a good combination of hyperparameters;
  - cross-validation can be done with a smaller number of splits or folds, but in this case the quality assessment becomes more noisy and the risk of choosing a non-optimal set of hyperparameters increases due to randomness of splitting
  - hyperparameters can be optimized sequentially (greedily) - one by one, and not to go through all the combinations; such a strategy does not always lead to an optimal set;
  - sort through not all combinations of hyperparameters, but a small number of randomly selected ones.

### Task

In this lab, we will learn to teach machine learning models, correctly set up experiments, select hyperparameters, compare and mix models. You are invited to solve the problem of binary classification, namely, to build an algorithm that determines whether the average person’s earnings exceed the threshold$50k.
 
More details about the features can be read [here] (http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names). The target attribute is written in the variable *> 50K, <= 50K *.

Download the data set * data.adult.csv *. To better understand what you are working with / whether you have correctly loaded the data, you can display the first few lines on the screen.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('./data.adult.csv')
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,">50K,<=50K"
0,34,Local-gov,284843,HS-grad,9,Never-married,Farming-fishing,Not-in-family,Black,Male,594,0,60,<=50K
1,40,Private,190290,Some-college,10,Divorced,Sales,Not-in-family,White,Male,0,0,40,<=50K
2,36,Local-gov,177858,Bachelors,13,Married-civ-spouse,Prof-specialty,Own-child,White,Male,0,0,40,<=50K
3,22,Private,184756,Some-college,10,Never-married,Sales,Own-child,White,Female,0,0,30,<=50K
4,47,Private,149700,Bachelors,13,Married-civ-spouse,Tech-support,Husband,White,Male,15024,0,40,>50K


Sometimes there are gaps in the data. The way to designate gaps is either prescribed in the data description, or at the skip place after reading the data, the value [NaN] appears (https://docs.scipy.org/doc/numpy-1.13.0/user/misc.html). For more information about working with passes in Pandas, you can read for example [here] (http://pandas.pydata.org/pandas-docs/stable/missing_data.html).
In this dataset, missing values are marked with a "?".

** (1 point) Task 1. ** Usually, after loading a dataset, some preprocessing is always necessary. In this case, it will be as follows:
 - Find all the features that have missing values. Remove from the sample all objects with gaps.
 - Save the target variable (the one we want to predict) into a separate variable, remove it from the dataset and convert to a binary format.
 - Please note that not all features are real (numeric). In the beginning we will work only with real features. Save them separately.

### (2 points) Training classifiers on real features

In this section, it will be necessary to work only with real attributes and the target variable.

In the beginning we will look at how the selection of hyperparameters on the grid works and how the quality of the partitioning affects the quality. Now and further we will consider 4 algorithms:
 - [kNN](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)
 - [DecisonTree](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier)
 - [SGD Linear Classifier](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html)
 - [RandomForest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

To begin with, the first three algorithms will choose one hyperparameter, which we will optimize:
  - kNN - number of neighbors (* n_neighbors *)
  - DecisonTree - tree depth (* max_depth *)
  - SGD Linear Classifier - optimized function (* loss *)
 
Leave the default values for the remaining hyperparameters. For the selection of hyperparameters, use the search on the grid, which is implemented in the class [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV). As a cross-validation scheme, use 5-Fold CV, which can be set using the class [KFoldCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html#sklearn.model_selection.KFold).

![](https://i.stack.imgur.com/YWgro.gif)



**(1.5 points) Task 2.** For each algorithm, select the optimal values of the specified hyperparameters. Build a graph of the average value of the quality of the cross-validation algorithm for a given value of the hyperparameter, which also display the confidence interval.

To obtain the value of quality on each fold, the average value of quality and other useful information, you can use the field [*cv results_*](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV)

Which algorithm has the highest average quality ?

Largest confidence interval ?

**(0.5 point) Task 3.** Now let's select the number of trees (* n_estimators *) in the RandomForest algorithm. As you know, in general, Random Forest does not retrain with an increase in the number of trees. Pick the number of trees starting from which the quality on cross-validation stabilizes. Note that to conduct this experiment, it is not necessary to train many random forests with different numbers of trees from scratch: train one random forest with the maximum interesting number of trees, and then consider subsets of trees of different sizes consisting of trees of the constructed forest (field [*estimators_*](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)). In further experiments, use the number of trees found.

It is difficult to apply the * GridSearchCV * class in this task, so it is suggested to write a cycle on the number of trees.

When learning algorithms, it is worth paying attention not only to their quality, but also how they work with data. In this problem, it turned out that some of the algorithms used are sensitive to the scale of features. To make sure that this could affect the quality, let's look at the meanings of the attributes themselves.

**(1 point) Task 4.** Look at the values of the signs * age *, * fnlwgt *, * capital-gain *. What is the feature of the data? Which of the considered algorithms can affect this? Can scaling affect the operation of these algorithms?

You can scale attributes, for example, in one of the following ways:
 - $x_{new} = \dfrac{x - \mu}{\sigma}$, where $\mu, \sigma$ — the mean and standard deviation of the characteristic value over the entire sample (see the function [scale](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.scale.html))
 - $x_{new} = \dfrac{x - x_{min}}{x_{max} - x_{min}}$, where $[x_{min}, x_{max}]$ — minimum interval of characteristic values

Similar scaling schemes are given in classes [StandardScaler](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler) and [MinMaxScaler](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html#sklearn.preprocessing.MinMaxScaler).
 
**(1 point) Task 5.** 

Scale all numerical attributes using one of the above methods and select the optimal values of the hyperparameters in the same way as above.

Has the quality of some algorithms changed and why?

**(1.5 point) Task 6.** Now go through several hyperparameters across the grid and find the optimal combinations (best average quality value) for each algorithm in this case:
  - KNN - number of neighbors (* n_neighbors *) and metric (* metric *)
  - DecisonTree - tree depth (* max_depth *) and partitioning criterion (* criterion *)
  - RandomForest - the partitioning criterion in trees (* criterion *) and the maximum number of considered features (* max_features *); use the trees found earlier
  - SGDClassifier - optimized function (* loss *) and * penalty *
 
Please note that this operation can be resource intensive and time consuming. How to optimize the selection of parameters on the grid is described in the section "Selection of hyperparameters of the model".

Which algorithm has the best quality?

**(1.5 points) Task 7.** Build for different algorithms graphics [learning curves](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.learning_curve.html), depicting the dependence of quality on test and training samples on the number of objects on which models are trained. Look at the behavior of the curves and answer the questions:
* Can the quality on the test sample decrease with an increase in the number of objects? And on the training? Why? 
* For what purposes can quality knowledge be used on the training part of the sample?
* Which algorithm is better trained on fewer objects?
* Can the addition of new objects significantly improve the quality of any of the algorithms or, with the existing data set for all algorithms, saturation occur?

### (1 point) Adding categorical features to models

So far we have not used non-numeric attributes that are in dataset. Let's see if we did the right thing and whether the quality of the models will increase after adding these attributes.

**(0.5 points) Task 8.** Transform all categorical features using the one-hot-encoding method (for example, you can do this using [pandas.get_dummies](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) or [DictVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html) / [OneHotEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) from sklearn).

Since after coding, signs turned out to be quite a lot, in this work we will not re-select the optimal hyperparameters for models taking into account new signs (although it would be better to do it). 

**(0.5 points) Task 9.** Add coded categorical to scaled real-time features and train algorithms with the best hyper-parameters found earlier. Did the addition of new signs increase the quality? Measure quality as before using a 5-fold CV. For this it is convenient to use the function [cross_val_score](http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.cross_val_score.html#sklearn.cross_validation.cross_val_score).

Is the best classifier now different from the best in the previous paragraph?

### (2 points) Mixing models (blending)When implementing their models, it is good practice to create sklearn-compatible classes. First, such an implementation will have a standard interface and will allow other people to teach the models you have implemented without serious consequences. Secondly, it is possible to use any sklearn package functionality that accepts a model as input, for example, the class * GridSearchCV *, * learning_curve * and others.

Create a classifier that is initialized with two arbitrary classifiers and the $ \ alpha $ parameter. During training, such a classifier should train both basic models, and at the stage of prediction, knead predictions of basic models according to the formula indicated above.

To create a custom classifier, you must inherit from the base classes.


In all the preceding paragraphs, we obtained many strong models that can be quite different in nature (for example, the method of the nearest neighbors and the random forest). Often in practice it is possible to increase the quality of prediction by mixing different models. Let's see if this approach really gives an increase in quality.

Choose from the constructed models of the two previous points two, which gave the highest quality on cross-validation (we denote them $ clf_1 $ and $ clf_2 $). Next, build a new classifier, whose answer on some object $ x $ will look like this:

$$result(x) = clf_1(x) * \alpha + clf_2(x) * (1 - \alpha)$$

where $ \alpha $ is a hyper-parameter of the new classifier.

**(1 point) Task 10.**
When implementing their models, it is good practice to create sklearn-compatible classes. First, such an implementation will have a standard interface and will allow other people to teach the models you have implemented without serious consequences. Secondly, it is possible to use any sklearn package functionality that accepts a model as input, for example, the class * GridSearchCV *, * learning_curve * and others.

Create a classifier that is initialized with two arbitrary classifiers and the $ \ alpha $ parameter. During training, such a classifier should train both basic models, and at the stage of prediction, knead predictions of basic models according to the formula indicated above.

To create a custom classifier, you must inherit from the base classes.
*[BaseEstimator](http://scikit-learn.org/stable/modules/generated/sklearn.base.BaseEstimator.html), [ClassifierMixin](http://scikit-learn.org/stable/modules/generated/sklearn.base.ClassifierMixin.html)* and implement methods*\_\_init\_\_, fit, predict and predict_proba*. Example sklearn-compatible classifier with comments can be found [here](http://scikit-learn.org/stable/developers/contributing.html#rolling-your-own-estimator) 

**(1 point) Task 11.** Select the value $ \ alpha $ for this classifier from the grid from 0 to 1. If the class is implemented correctly, then you can use * GridSearchCV *, as is the case with conventional classifiers.

Draw on the graph the average quality of the folds and the confidence interval depending on $ \ alpha $.

Did this approach increase in quality compared to models that were trained separately? Explain why even simple blending of models can influence the final quality?

## (1 point) Models comparison

![](http://cdn.shopify.com/s/files/1/0870/1066/files/compare_e8b89647-3cb6-4871-a976-2e36e5987773.png?1750043340268621065)

After many models have been built, the right continuation is to compare them with each other. At the visualization workshop, you were shown how to build a “mustache box” (span diagram). We use it to compare the algorithms with each other.

**(1 point) Task 12.** For each type of classifier (kNN, DecisionTree, RandomForest, SGD classifier), as well as a mixed model, choose the one that gave the best quality for cross-validation and build a span diagram. All classifiers should be displayed on the same graph.
 
Make general summary conclusions about the classifiers in terms of their work with the signs and the complexity of the model itself (what kind of hyperparameters the model has, whether changing the value of the hyperparameter greatly affects the quality of the model).